# AceleraDev DataScience 

## Setup

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data




In [2]:
#lendo os pacotes
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [4]:
df = pd.read_csv('../data/train.csv')

In [6]:
y_train = df['SalePrice']
X_train = df.drop(columns = 'SalePrice')

In [8]:
X_test = pd.read_csv('../data/test.csv')
y_test = pd.read_csv('../data/sample_submission.csv')

## Analysis

### Selecao por completude

In [9]:
#Criando um dataframe auxliar para analisar a consistencia das variaveis
cons = pd.DataFrame({'colunas' : df.columns,
                    'tipo': df.dtypes,
                    'missing' : df.isna().sum(),
                    'size' : df.shape[0],
                    'unicos': df.nunique()})
cons['percentual'] = round(cons['missing'] / cons['size'],2)

### Removendo colunas com dados missing

In [10]:
print('Contagem de colunas com ATÉ 20% de dados faltantes', cons[cons.percentual < 0.2].shape[0])
print('Contagem de colunas com 0% de dados faltantes',  cons[cons.percentual == 0].shape[0])

Contagem de colunas com ATÉ 20% de dados faltantes 76
Contagem de colunas com 0% de dados faltantes 63


In [11]:
cons[cons.percentual == 0]['tipo'].value_counts()

int64     35
object    28
Name: tipo, dtype: int64

In [12]:
cons['completa'] = ['completa' if x == 0 else 'faltante' for x in cons['percentual']]

In [21]:
mantem = list(cons[cons['completa'] == 'completa']['colunas'])
df = df[mantem]

In [ ]:
df.drop

In [49]:
colunas_numericas = list(cons[((cons['tipo'] != 'object') &
                              (cons['completa'] == 'completa'))]['colunas'])
colunas_numericas.remove('SalePrice')

In [27]:
colunas_categoricas = list(cons[((cons['tipo'] == 'object') &
                              (cons['completa'] == 'completa'))]['colunas'])

## Categoricas

### Label Encoding

In [26]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition', 'SalePrice'],
      dtype='object')

### Label encoder

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
encoding = LabelEncoder()

In [33]:
encoding.fit_transform(df['TotRmsAbvGrd'])

array([6, 4, 4, ..., 7, 3, 4])

### Dummies

In [36]:
df.shape

(1460, 63)

In [35]:
pd.get_dummies(df).shape

(1460, 222)

### Catboost

In [37]:
import category_encoders as ce

In [42]:
catb = ce.CatBoostEncoder(cols=colunas_categoricas)

In [45]:
catb.fit_transform(X_train[colunas_categoricas], y_train)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Foundation,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,...,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890
1,194710.597945,194710.597945,194710.597945,194710.597945,194710.597945,180921.195890,194710.597945,180921.195890,180921.195890,194710.597945,...,180921.195890,194710.597945,194710.597945,194710.597945,194710.597945,180921.195890,194710.597945,194710.597945,194710.597945,194710.597945
2,190307.065297,190307.065297,180921.195890,190307.065297,190307.065297,194710.597945,190307.065297,194710.597945,194710.597945,190307.065297,...,194710.597945,190307.065297,190307.065297,190307.065297,190307.065297,194710.597945,190307.065297,190307.065297,190307.065297,190307.065297
3,198605.298973,198605.298973,202210.597945,198605.298973,198605.298973,180921.195890,198605.298973,180921.195890,204307.065297,198605.298973,...,180921.195890,198605.298973,180921.195890,198605.298973,198605.298973,204307.065297,198605.298973,198605.298973,198605.298973,180921.195890
4,186884.239178,186884.239178,181473.731963,186884.239178,186884.239178,181210.597945,186884.239178,180921.195890,188230.298973,186884.239178,...,204307.065297,186884.239178,198605.298973,186884.239178,186884.239178,188230.298973,186884.239178,186884.239178,186884.239178,198605.298973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,191008.321880,181105.930480,164644.954610,180153.548734,180925.939654,176887.998279,179927.498691,192929.458176,184477.799519,181144.751697,...,225239.593811,181998.639885,214852.720535,186175.352826,186781.834858,139864.757430,183429.208693,186423.144608,173349.107835,175152.475038
1456,190994.377348,181101.722396,164656.185679,180149.608712,180921.869640,176886.198471,179923.925450,188651.728711,184470.259503,181140.490427,...,149775.505057,181993.728559,142294.935037,186167.147721,186772.989637,139912.625607,145447.457932,186414.600745,173350.413921,175152.347444
1457,191010.918360,181121.624791,164705.312238,180172.412678,180941.827176,176917.735425,179945.719707,208946.709723,184490.553415,181160.490087,...,151696.032648,182013.368300,214798.938186,186184.633306,186790.414250,212089.378113,183423.001617,186432.228099,173379.385926,175181.484278
1458,191076.561040,181180.384856,164815.479649,180238.311600,181000.509051,177002.970691,180008.394783,146019.507537,184555.691974,181219.589471,...,149870.646439,182072.574069,157059.517483,186243.515540,122609.619105,212182.229003,183484.132594,186492.024792,173452.940913,175257.773764


## Numericas

### Standard Scaler

In [46]:
from sklearn.preprocessing import StandardScaler

In [47]:
std = StandardScaler()

In [50]:
std.fit_transform(X_train[colunas_numericas])

array([[-1.73086488,  0.07337496, -0.20714171, ..., -0.08768781,
        -1.5991111 ,  0.13877749],
       [-1.7284922 , -0.87256276, -0.09188637, ..., -0.08768781,
        -0.48911005, -0.61443862],
       [-1.72611953,  0.07337496,  0.07347998, ..., -0.08768781,
         0.99089135,  0.13877749],
       ...,
       [ 1.72611953,  0.30985939, -0.14781027, ...,  4.95311151,
        -0.48911005,  1.64520971],
       [ 1.7284922 , -0.87256276, -0.08016039, ..., -0.08768781,
        -0.8591104 ,  1.64520971],
       [ 1.73086488, -0.87256276, -0.05811155, ..., -0.08768781,
        -0.1191097 ,  0.13877749]])

### MinMaxScaler

In [51]:
from sklearn.preprocessing import MinMaxScaler

In [52]:
minmax = MinMaxScaler()

In [53]:
minmax.fit_transform(X_train[colunas_numericas])

array([[0.00000000e+00, 2.35294118e-01, 3.34198042e-02, ...,
        0.00000000e+00, 9.09090909e-02, 5.00000000e-01],
       [6.85400960e-04, 0.00000000e+00, 3.87950174e-02, ...,
        0.00000000e+00, 3.63636364e-01, 2.50000000e-01],
       [1.37080192e-03, 2.35294118e-01, 4.65072799e-02, ...,
        0.00000000e+00, 7.27272727e-01, 5.00000000e-01],
       ...,
       [9.98629198e-01, 2.94117647e-01, 3.61868705e-02, ...,
        1.61290323e-01, 3.63636364e-01, 1.00000000e+00],
       [9.99314599e-01, 0.00000000e+00, 3.93418869e-02, ...,
        0.00000000e+00, 2.72727273e-01, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 4.03701886e-02, ...,
        0.00000000e+00, 4.54545455e-01, 5.00000000e-01]])

### MinAbsScaler

In [54]:
from sklearn.preprocessing import MaxAbsScaler

In [55]:
maxabs = MaxAbsScaler()

In [56]:
maxabs.fit_transform(X_train[colunas_numericas])

array([[6.84931507e-04, 3.15789474e-01, 3.92575902e-02, ...,
        0.00000000e+00, 1.66666667e-01, 9.99004975e-01],
       [1.36986301e-03, 1.05263158e-01, 4.46003391e-02, ...,
        0.00000000e+00, 4.16666667e-01, 9.98507463e-01],
       [2.05479452e-03, 3.15789474e-01, 5.22660224e-02, ...,
        0.00000000e+00, 7.50000000e-01, 9.99004975e-01],
       ...,
       [9.98630137e-01, 3.68421053e-01, 4.20079444e-02, ...,
        1.61290323e-01, 4.16666667e-01, 1.00000000e+00],
       [9.99315068e-01, 1.05263158e-01, 4.51439058e-02, ...,
        0.00000000e+00, 3.33333333e-01, 1.00000000e+00],
       [1.00000000e+00, 1.05263158e-01, 4.61659969e-02, ...,
        0.00000000e+00, 5.00000000e-01, 9.99004975e-01]])